### Creating a basic LSTM Sequence-to-Sequence model for translating Japanese to English

In [1]:
import os
import json
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
# get JSON file of sentence pairs
benchmark_directory = os.getcwd()
os.chdir(os.path.join(benchmark_directory, './..'))
sentences_file = open('./subtitle_corpus.json', 'r')
sentences_json = json.load(sentences_file)

In [17]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
# restrict to subsample before testing local runtimes
num_samples = 2000  # Number of samples to train on.
sentence_subset = sentences_json[:num_samples]

In [18]:
# handle empty data
def xstr(s):
    if s is None:
        return ''
    return str(s)

In [19]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

for sentence in sentence_subset[: min(num_samples, len(sentence_subset) - 1)]:
    input_text, target_text = sentence['j'], sentence['e']
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + xstr(target_text) + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [20]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 1999
Number of unique input tokens: 1559
Number of unique output tokens: 49
Max sequence length for inputs: 82
Max sequence length for outputs: 173


In [21]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [22]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [23]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [24]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [25]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [26]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/100
25/25 [==============================] - 20s 798ms/step - loss: 1.1980 - accuracy: 0.7879 - val_loss: 0.7979 - val_accuracy: 0.8297
Epoch 2/100
25/25 [==============================] - 21s 822ms/step - loss: 0.8857 - accuracy: 0.8201 - val_loss: 0.7491 - val_accuracy: 0.8305
Epoch 3/100
25/25 [==============================] - 20s 819ms/step - loss: 0.7597 - accuracy: 0.8199 - val_loss: 0.6619 - val_accuracy: 0.8311
Epoch 4/100
25/25 [==============================] - 20s 817ms/step - loss: 0.6896 - accuracy: 0.8210 - val_loss: 0.6429 - val_accuracy: 0.8318
Epoch 5/100
25/25 [==============================] - 21s 833ms/step - loss: 0.7089 - accuracy: 0.8216 - val_loss: 1.2234 - val_accuracy: 0.8330
Epoch 6/100
25/25 [==============================] - 21s 834ms/step - loss: 0.6986 - accuracy: 0.8222 - val_loss: 0.6082 - val_accuracy: 0.8327
Epoch 7/100
25/25 [==============================] - 21s 838ms/step - loss: 0.6379 - accuracy: 0.8234 - val_loss: 0.5902 - val_accuracy:

25/25 [==============================] - 21s 840ms/step - loss: 0.3246 - accuracy: 0.9027 - val_loss: 0.4022 - val_accuracy: 0.8845
Epoch 58/100
25/25 [==============================] - 21s 845ms/step - loss: 0.3209 - accuracy: 0.9035 - val_loss: 0.4033 - val_accuracy: 0.8841
Epoch 59/100
25/25 [==============================] - 21s 840ms/step - loss: 0.3163 - accuracy: 0.9049 - val_loss: 0.4082 - val_accuracy: 0.8832
Epoch 60/100
25/25 [==============================] - 21s 843ms/step - loss: 0.3122 - accuracy: 0.9059 - val_loss: 0.4079 - val_accuracy: 0.8830
Epoch 61/100
25/25 [==============================] - 21s 846ms/step - loss: 0.3172 - accuracy: 0.9047 - val_loss: 0.4082 - val_accuracy: 0.8835
Epoch 62/100
25/25 [==============================] - 21s 847ms/step - loss: 0.3097 - accuracy: 0.9073 - val_loss: 0.4107 - val_accuracy: 0.8830
Epoch 63/100
25/25 [==============================] - 21s 851ms/step - loss: 0.3117 - accuracy: 0.9063 - val_loss: 0.4113 - val_accuracy: 0.882

In [27]:
# Save model
model.save('s2s.h5')

In [28]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [29]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [30]:
for seq_index in range(15):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: あなたは戻ったのね ハロルド?
Decoded sentence: i want to the car mecatah way.

-
Input sentence: 俺の相手は シャークだ。
Decoded sentence: i want to the car mecatah way.

-
Input sentence: 引き換えだ ある事とある物の
Decoded sentence: i want to the car mecatah way.

-
Input sentence: もういいよ ごちそうさま ううん
Decoded sentence: i want to the car mecatah way.

-
Input sentence: もう会社には来ないでくれ 電話もするな
Decoded sentence: i want to the car mecatah way.

-
Input sentence: きれいだ。
Decoded sentence: i want to the car mecatah way.

-
Input sentence: 連れて行け 殺しそうだ わかったか?
Decoded sentence: i want to the car mecatah way.

-
Input sentence: 殺したのか!
Decoded sentence: i want to the car mecatah way.

-
Input sentence: わぁ~! いつも すみません。 いいのよ~。
Decoded sentence: i don't know have anyeding.

-
Input sentence: カンパニーの元社員が
Decoded sentence: i want to the car mecatah way.

-
Input sentence: じゃアイツラはどこ?
Decoded sentence: i want to the car mecatah way.

-
Input sentence: 相手を陥れるとか...
Decoded sentence: i don't know have anyedidn

-
Input sentence: 必要の